# Introducere
Acest proiect am incercat sa arat tot ce cunosc si sa il fac structurat.
Nu a fost greu sa ma descurc cu acest task, l-am facut plus minus in timp de 2 zile dupa trimiterea task-ului, dar nu l-am trimis deodata pentru ca nu am avut timp destul sa il pun in **jupyter notebook**.

Codul la reteaua neuronala am facut dar sa o antrenez nu am reusit, prea greu mergea procesul.
In antrenare folosesc metoda **backpropagation**(Nu pana la capat am inteles-o pentru ca nu stiu analiza matematica, pana cand).

Pentru a face acest proiect am studiat informatie despre inteligenta artificiala dintr-un articol de pe **[habr](https://habr.com/ru/articles/313216/)**.

Unica ce a fost straniu asta ceea ca in examenarea mea pentru cursuri de inteligenta artificiala, mi-ati dat sa fac o inteligenta artificiala.

# main.py
Pornirea la programe si input la date

In [ ]:
import asyncio
import warnings

from app import App

if __name__ == "__main__":
    warnings.simplefilter("ignore", RuntimeWarning)
    config_path = 'data/config.json'

    while True:
        run_type = input('\n1) Scrieti "learn" pentru a invata inteligenta artificiala\n2) Scrieti lista de date separata prin viruga(exemplu: "1,24,3,4") pentru a utiliza AI\n>>> ')

        app = App(config_file=config_path, get_weighting=True)
        asyncio.run(app.run(type=run_type))

# app.py
Aplicatia principala care lucreaza asincron(*acesta am facut ca sa am posibilitatea de a adauga noi functii fara sa incurc la procesul de invatare spre exemplu*)


In [ ]:
import asyncio

import data_operation
import Katwork

class App(data_operation.File):

    def __init__(self, 
                 config_file: str='config.json', 
                 get_weighting: bool=False
                 ) -> None:
        '''
            1. config_file: drumul spre fisierul config.json
            2. get_weighting: False/True zice daca sa ia sau nu cantarurile salvate in config.json pentru reteaua neuronala.
        '''
        self.config_path = config_file
        self.config = self.get_json(file=self.config_path)
        self.nn_weighting = self.config['neuralnetwork']['weighting']

        self.sigmoid = Katwork.NeuralNetwork(length_input_output_layout=(8, 1), 
                                             neurons_layout=[10, 10, 10], 
                                             displacement_neuron=False, 
                                             function='sigmoid') #Genereaza reteaua neuronala
        
        self.get_weighting = get_weighting

    async def run(self,
                  type: str='learn'
            ) -> None:
        '''
            type: 'learn' sau etc. zice daca sa invete reteaua neuronala ori nu.
        '''
        if self.get_weighting:
            returned = await self.load_weighting()
            if returned is not None:
                print(returned)

        if type == 'learn':
            task = asyncio.create_task(self.learn())

            await task
        else:
            task = asyncio.create_task(self.input(input_data=[float(e) for e in type.split(',')], binar=True))

            print((await task)[0])

    async def load_weighting(self
                       ) -> str|None:
        if self.nn_weighting != None: self.sigmoid.neuralnetwork = self.nn_weighting
        else: return -1

    async def input(self, 
              input_data: list,
              binar: bool=False
              ) -> tuple:
        '''
            input_data: datele de intrare pentru reteaua neuronala.
            binar: False/True zice sa returneze output-ul retelei neuronale in forma de procente ori ca 1/0
        '''
        f = lambda x: 1 if x >= 0.5 else 0
        output_data = await self.sigmoid.run(input_layout=input_data)

        return output_data if not binar else ([f(o) for o in output_data[0]], output_data[1])

    async def learn(self
              ) -> None:
        data = tuple([[int(e) if i >= 8 or e == '0' else float(e)
                       for i, e in enumerate(example[1:])] 
                      for example in self.get_csv(file=self.config['path']['nn_data_learn'])])
        
        error = await self.sigmoid.backpropagation(example_list=data, maximal_era=100, learn_speed=0.7, moment=0.3)

        self.config['neuralnetwork']['error'] = error
        self.config['neuralnetwork']['weighting'] = self.sigmoid.neuralnetwork

        self.put_json(f'{self.config_path}', self.config)

# data_operation.py
Operarea cu fisierele **.csv** sau **.json**

In [3]:
import csv
import json

class File:

    def get_csv(self, 
                file: str
                ) -> list:
        with open(file, newline='') as csv_file:
            data_file = [row for row in csv.reader(csv_file, delimiter=',', quotechar='|')][1:]

        return data_file

    def get_json(self, 
                 file: str
                 ) -> dict:
        with open(file, 'r') as json_file:
            data_file = json.loads(json_file.read())
        
        return data_file
    
    def put_json(self, 
                 file: str, 
                 data: dict
                 ) -> None:
        with open(file, 'w') as json_file:
            json.dump(data, json_file, indent=4)

# Katwork.py
* Generarea retelei neuronale
* Pornirea retelei neuronale
* Invatarea retelei neuronale

In [4]:
import asyncio
from random import uniform
import math
from copy import deepcopy

class NeuralNetwork:

    def __init__(self, 
                 length_input_output_layout: tuple, 
                 neurons_layout: tuple=[], 
                 displacement_neuron: bool=False, 
                 function: str='liniar-function'
                 ) -> None:
        '''
            1. length_input_output_layout: 1 element asta e cifra care arata cati neuroni sunt in input layout. 2 element e cifra care arata cati neuroni sunt in ouput layout.
            2. neurons_layout: Cate elemente sunt in lista asta e cate coloane de neuroni sunt. Elementul este cifra care arata cate neuroni e in coloana.
            3. displacement_neuron: Neuronul de deplasare.
            4. function: dinumirea la functie care sa fie utilizata(sigmoid, hyperbolic-tangent, liniar-function).
        '''
        self.f = lambda x: (1 / (1 + math.e**(-x)) if function.lower() == 'sigmoid' else (math.e**(2*x) - 1) / (math.e**(2*x) + 1) if function.lower() == 'hyperbolic-tangent' else x if function.lower() == 'linear-function' else None)
        self.length_i_o = length_input_output_layout
        self.displacement_neuron = displacement_neuron
        self.neuralnetwork = []

        if self.f == None: return 'error'

        #Generate hidden layouts
        for l_i, l in enumerate(neurons_layout):
            layout = [] 
            for n in range(l):
                neuron = []
                for s in range((neurons_layout[l_i-1] if l_i > 0 else self.length_i_o[0]) + (1 if self.displacement_neuron else 0)): neuron.append(uniform(-0.5, 1))

                layout.append(neuron)

            self.neuralnetwork.append(layout)

        #Generate output layout
        layout = []
        for n in range(self.length_i_o[1]):
            neuron = []
            for w in range((len(self.neuralnetwork[-1]) if len(self.neuralnetwork) > 0 else self.length_i_o[0]) + (1 if self.displacement_neuron else 0)): neuron.append(uniform(-0.5, 1))

            layout.append(neuron)
        
        self.neuralnetwork.append(layout)

    async def MSE(self, 
            all_difference: int|float, 
            example_range: int
            ) -> float:
        '''
            1. all_difference: calculul la toate exemplele: (a_1 - i_1)^2 + (a_2 - i_2)^2 ... (a_n - i_n)^2
            2. example_length: numarul de exemple.
        '''
        return all_difference / example_range
    
    async def root_MSE(self, 
                 all_difference: int|float, 
                 example_range: int
                 ) -> float:
        '''
            1. all_difference: calculul la toate exemplele: (a_1 - i_1)^2 + (a_2 - i_2)^2 ... (a_n - i_n)^2
            2. example_length: numarul de exemple.
        '''
        return math.sqrt(all_difference / example_range)
    
    async def update_w(self
                 ) -> None:
        for l_i, l in enumerate(self.neuralnetwork):
            for n in range(len(l)):
                for w in range(len(l[n])): self.neuralnetwork[l_i][n][w] = uniform(-0.5, 1)
    
    async def backpropagation(self, 
                        example_list: tuple|list, 
                        maximal_era: int, 
                        learn_speed: float|int=0.7, 
                        moment: int|float=0.3
                        ) -> list:
        '''
            1. example: lista cu exemple dupa care NN se invata. Fiecare element din lista asta e un exemplu, un exemplu consta din lista care are in ea atatia neuroni de intrare cat ati indicat la crearea NN si dupa urmeaza neuronii de iesire, tot cat ati indicat la crearea la NN(Rezultatul croect).
            2. maximal_era: era maximala pana la care poate sa se invete.
            3. learn_speed: Rata cu care se algoritmul de invatare ajusteaza parametrii sai in functie de datele de antrenament. O valoare mai a vitedezei de invatare poate duce la convergenta mai rapida, dar poate provoca si oscilatii sau divergenta.
            4. moment: Rata care accelereaza convergenta si care eviti minimul local.
            5. show_data: arata datele despre evolutie, ce secol sau mileniu.
        '''

        back_neuralnetwork = deepcopy(self.neuralnetwork)
        back_neuralnetwork.reverse()
        back_neuralnetwork.append([[] for n in range(self.length_i_o[0])])

        gradient = deepcopy(back_neuralnetwork)
        w_shift = deepcopy(back_neuralnetwork)

        delta1 = lambda output_ideal, output_actual: (output_ideal - output_actual) * ((1 - output_actual) * output_actual) #Formula la delta pentru output
        delta2 = lambda neuron, sum_wd: ((1 - neuron) * neuron) * sum_wd #Formula la delta pentru hidden
        grad = lambda neuron, delta: delta * neuron
        shift = lambda E, grad, A, last_shift: E*grad + A*last_shift

        for era in range(maximal_era):
            error = [0 for i in range(self.length_i_o[1])]

            for iteration, example in enumerate(example_list):
                output = await self.run(input_layout=example[:self.length_i_o[0]], get_layout=True)

                actual_output = deepcopy(output[0])
                all_data = deepcopy(output[1])
                all_data.reverse()
                
                #Output operation
                delta_output_layout = [0 for d in range(self.length_i_o[1])]
                for i, n in enumerate(actual_output):
                    error[i] += (example[self.length_i_o[0]:][i] - n)**2
                    delta_output_layout[i] = delta1(output_ideal=example[self.length_i_o[0]:][i], output_actual=n)
                
                #Hidden and Input operation
                copy_nn = deepcopy(back_neuralnetwork)
                for i, l in enumerate(copy_nn):
                    if i == len(copy_nn)-1: break

                    delta_hidden_layout = [0 for d in range(len(all_data[i+1]))] 
                    for i2, n in enumerate(copy_nn[i+1]):
                        sum_wd = 0
                        for i3, n2 in enumerate(l):
                            sum_wd += n2[i2] * delta_output_layout[i3]
                            
                            gradient[i][i3][i2] = grad(all_data[i+1][i2], delta_output_layout[i3])
                            w_shift[i][i3][i2] = shift(learn_speed, gradient[i][i3][i2], moment, 0 if iteration == 0 else w_shift[i][i3][i2])
                            back_neuralnetwork[i][i3][i2] += w_shift[i][i3][i2]
                        
                        delta_hidden_layout[i2] = delta2(all_data[i+1][i2], sum_wd)

                    delta_output_layout = delta_hidden_layout

                neuralnetwork = deepcopy(back_neuralnetwork)
                neuralnetwork.pop(-1)
                neuralnetwork.reverse()
                self.neuralnetwork = neuralnetwork
                        
            for i in range(self.length_i_o[1]):
                error[i] = await self.MSE(all_difference=error[i], example_range=len(example_list))

            print(f'---------\nEra: {era}\nError: {error}')
        
        return error

    async def run(self, 
            input_layout: list, 
            get_layout: bool=False
            ) -> tuple[list[float]|None]:
        '''
            1. input_layout: lista cu parametrele de intrare(cati neuroni de intrare sunt atitea parametre trebuie sa fie, un element arata starea la un neuron de intrare).
        '''

        input_data = input_layout + ([1] if self.displacement_neuron else [])
        all_data = [input_data]
        for l in self.neuralnetwork:
            layout_output_data = []
            for n in l:
                result = 0
                for i, w in zip(input_data, n):
                    result += self.f(i) * w

                layout_output_data.append(self.f(result) if result != 0 else result)
            input_data = layout_output_data + ([1] if self.displacement_neuron else [])
            all_data.append(input_data)

        return input_data[:-1] if self.displacement_neuron else input_data, all_data if get_layout else None